In [1]:
import sys, os
# import google.generativeai as genai
sys.path.append(os.path.abspath(os.path.join('.', 'secret')))
from secret_info import google_genai_api

GOGGLE_API_KEY = google_genai_api
# genai.configure(api_key = GOGGLE_API_KEY)

## LLMs wrappers

In [21]:
from langchain_google_genai import ChatGoogleGenerativeAI, HarmBlockThreshold, HarmCategory
chat = ChatGoogleGenerativeAI(model="gemini-pro", google_api_key=GOGGLE_API_KEY, convert_system_message_to_human=False, temperature=0.7, safety_settings={
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
    },)
chat.invoke("explain how a car works in one sentence").content

'A car is a machine that converts the energy stored in gasoline into motion by burning it in an engine, which turns wheels.'

In [3]:
# Same as above
from langchain_google_genai import GoogleGenerativeAI
llm = GoogleGenerativeAI(model="gemini-pro", google_api_key=GOGGLE_API_KEY, convert_system_message_to_human=False, temperature=0.7)
llm.invoke("explain large language models in one sentence")

'Large language models are powerful artificial intelligence systems that can understand and generate human-like text, translate languages, write different kinds of creative content, and perform various language-related tasks.'

## CHAT MODEL

In [10]:
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
chat = ChatGoogleGenerativeAI(model="gemini-pro", google_api_key=GOGGLE_API_KEY, convert_system_message_to_human=False, temperature=0.7)
messages = [
    SystemMessage(content="You are an expert data scientist"),
    HumanMessage(content="Write a Python script that trains a neural network on simulated data ")
]
response = chat.invoke(input=messages)
print(response.content,end='\n')

```python
import numpy as np
import tensorflow as tf

# Generate simulated data
num_samples = 1000
X = np.random.rand(num_samples, 10)
y = np.sin(X[:, 0])

# Create a neural network model
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(10, activation='relu', input_shape=(10,)),
  tf.keras.layers.Dense(1)
])

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Train the model
model.fit(X, y, epochs=100)

# Evaluate the model
loss, accuracy = model.evaluate(X, y)
print('Loss:', loss)
print('Accuracy:', accuracy)
```


## PROMPTS

In [12]:
#Combined with https://python.langchain.com/v0.1/docs/integrations/llms/google_ai/ 
from langchain import PromptTemplate
from langchain_google_genai import GoogleGenerativeAI

llm = GoogleGenerativeAI(model="gemini-pro", google_api_key=GOGGLE_API_KEY)

template = """
You are an expert data scientist with an expertise in building deep learning models. 
Explain the concept of {concept} in 200 words
"""

prompt = PromptTemplate(
    input_variables=["concept"],
    template=template,
)

chain = prompt | llm

question = "How much is 2+2?"
print(chain.invoke({"concept": 'business analytics'}))

Business analytics is the process of using data and analytics to gain insights into business operations and make better decisions. It involves collecting, cleaning, and analyzing data to identify trends, patterns, and relationships. Business analytics can be used to improve a variety of business processes, including:

* **Customer relationship management (CRM):** Business analytics can be used to track customer behavior, preferences, and satisfaction. This information can be used to improve marketing campaigns, personalize customer experiences, and increase customer loyalty.
* **Supply chain management:** Business analytics can be used to optimize inventory levels, reduce lead times, and improve delivery efficiency. This can lead to lower costs and improved customer service.
* **Financial planning and analysis:** Business analytics can be used to forecast revenue and expenses, identify financial risks, and make investment decisions. This can help businesses make more informed decisions

In [13]:

# Define a second prompt 

second_prompt = PromptTemplate(
    input_variables=["ml_concept"],
    template="Turn the concept description of {ml_concept} and explain it to me like I'm five in 500 words",
)
chain_two = second_prompt | llm


In [15]:
from langchain.chains import SimpleSequentialChain
overall_chain = SimpleSequentialChain(chains=[chain, chain_two])

# Run the chain specifying only the input variable for the first chain.
explanation = overall_chain.run("autoencoder")
print(explanation)

KeyError: 'chains'

In [16]:
SimpleSequentialChain??

Init signature:
SimpleSequentialChain(
    *,
    name: Optional[str] = None,
    memory: Optional[langchain_core.memory.BaseMemory] = None,
    callbacks: Union[List[langchain_core.callbacks.base.BaseCallbackHandler], langchain_core.callbacks.base.BaseCallbackManager, NoneType] = None,
    verbose: bool = None,
    tags: Optional[List[str]] = None,
    metadata: Optional[Dict[str, Any]] = None,
    callback_manager: Optional[langchain_core.callbacks.base.BaseCallbackManager] = None,
    chains: List[langchain.chains.base.Chain],
    strip_outputs: bool = False,
    input_key: str = 'input',
    output_key: str = 'output',
) -> None
Source:        
class SimpleSequentialChain(Chain):
    """Simple chain where the outputs of one step feed directly into next."""

    chains: List[Chain]
    strip_outputs: bool = False
    input_key: str = "input"  #: :meta private:
    output_key: str = "output"  #: :meta private:

    class Config:
        """Configuration for this pydantic object."""

